# ETL process: transfering xml files to json/parquet format

In [1]:
import pyspark
from pyspark.sql import Row
from collections import OrderedDict
from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType,StringType,IntegerType,StructField
sc = pyspark.SparkContext(appName="DBstackex")
from pyspark.sql import SQLContext
# from pyspark.sql.types import *
sqlContext = SQLContext(sc)
# import pandas
import xml.etree.ElementTree as ET
from lxml import etree

In [2]:
def convert_to_row(d: dict) -> Row:
    dd={}
    for k in col_list:
        if k in d:
            dd[k] = d[k]
        else:
            dd[k] = None
    return Row(**OrderedDict(sorted(dd.items())))

In [3]:
xml_file=sc.textFile('data/input/Posts.xml')
tags=xml_file.filter(lambda l: l.strip().startswith("<row"))
elements = tags.map(lambda l: ET.fromstring(l))
keys_values = elements.map(lambda l: l.attrib)
# keys_values.take(3)

In [4]:
col_list = list(keys_values.flatMap(lambda d: d.keys()).distinct().toLocalIterator())

In [5]:
col_list

['OwnerUserId',
 'ClosedDate',
 'LastEditDate',
 'CommunityOwnedDate',
 'CommentCount',
 'Body',
 'ParentId',
 'Id',
 'AcceptedAnswerId',
 'Tags',
 'AnswerCount',
 'LastEditorDisplayName',
 'ViewCount',
 'PostTypeId',
 'CreationDate',
 'LastActivityDate',
 'OwnerDisplayName',
 'Score',
 'LastEditorUserId',
 'FavoriteCount',
 'Title']

In [6]:
df = keys_values.map(convert_to_row)
# df = df.toDF(samplingRatio = 0.1)
df=sqlContext.createDataFrame(df, samplingRatio = 0.1)

In [8]:
df.coalesce(1).toJSON().saveAsTextFile("./data/results/Posts")

In [ ]:
df.coalesce(1).write.csv('data/results/PostHistory.csv')

In [7]:
df.coalesce(1).write.parquet("data/results/Posts_parquet")